In [ ]:
# | default_exp wb_clim

In [ ]:
# | hide
from nbdev.showdoc import *
from fastcore.test import *
from fastcore.utils import *

In [ ]:
# | export
import collections
from typing import Dict
from pandera.typing import DataFrame
from sureau_ecos_py.climate_utils import compute_vpd_from_t_rh

from sureau_ecos_py.create_simulation_parameters import (
    create_simulation_parameters,
)

from sureau_ecos_py.create_climate_data import create_climate_data
from sureau_ecos_py.create_modeling_options import create_modeling_options



In [ ]:
# | export

def new_wb_clim(
    climate_data:DataFrame, # Dataframe created using the `create_climate_data` function
    year:int, # Year, __No definition found__
    day_of_year:int # Day of the year, __No definition found__


) -> Dict:  # Dictionary containing parameters to run the model
    "Create a list with all necessary daily climate values to run SureauR from climate_data"

    # Assert parameters ---------------------------------------------------------

    # Year
    assert isinstance(
        year, int
    ) and 3000 >= year > 0,"year must be a integer value between 0-3000"

    # Day of year
    assert isinstance(
        day_of_year, int
    ) and 367 >= day_of_year >= 1, "day_of_year must be a integer value between 1-365"

    # Create wb_clim dictionary -------------------------------------------------

    # Check that year and day_of_year are present inside the dataframe
    if year in climate_data['year'].values and day_of_year in climate_data['day_of_year'].values:

        # Get row index in climate frame based on year and doy
        row_index = climate_data[(climate_data['year'] == year) & (climate_data['day_of_year'] == day_of_year)].index[0]

        # Transfrom row to a dictionary with params
        wb_clim_dict = collections.defaultdict(list, dict(climate_data.loc[row_index]))

    else:
        raise ValueError(
            f"year:{year} and/or day_of_year:{day_of_year} not found in climate dataframe"
        )

    # Add parameters to dictionary ----------------------------------------------
    wb_clim_dict['net_radiation'] = float("NAN")
    wb_clim_dict['etp'] = float("NAN")
    wb_clim_dict['vpd'] = compute_vpd_from_t_rh(relative_humidity = wb_clim_dict['RHair_mean'],
                                                temperature= wb_clim_dict['Tair_mean']
                                                )

    # Add Temperature from previous and next days





    return wb_clim_dict



    #print(index)

    #return wb_clim_dict





#### __Example: __

In [ ]:
simulation_parameters_dict = create_simulation_parameters(
    main_dir="./sample_data",
    start_year_simulation=1991,
    end_year_simulation=1991,
    output_type=None,
    output_path="./sample_data",
    overwrite=True,
    resolution_output="subdaily",
)

In [ ]:
modeling_options_dict = create_modeling_options(
    time_step_for_evapo=2,
    reset_swc=True,
    avoid_water_soil_transfer=True,
    constant_climate=False,
    defoliation=True,
    soil_evapo=True,
    threshold_mortality=51,
    etp_formulation="pt",
    rn_formulation="linear",
    comp_options_for_evapo="custom",
    stomatal_reg_formulation="turgor",
    transpiration_model="jarvis",
    numerical_scheme="implicit",
    pedo_transfer_formulation="vg",
)

In [ ]:
climate_dataframe = create_climate_data(
    modeling_options=modeling_options_dict,
    simulation_parameters=simulation_parameters_dict,
    file_path="./sample_data/climat_example.csv",
    sep=";",
)

365 days were selected in the input climate file, covering the period: 1991 - 1991


In [ ]:
new_wb_clim(
    climate_data = climate_dataframe,
    year = 1991,
    day_of_year = 1
)


defaultdict(list,
            {'DATE': Timestamp('1991-01-01 00:00:00'),
             'Tair_min': 2.6,
             'Tair_max': 7.3,
             'Tair_mean': 4.95,
             'RG_sum': 3.2,
             'RHair_min': 95.16042122,
             'RHair_max': 95.16042122,
             'RHair_mean': 95.16042122,
             'WS_mean': 3.1,
             'PPT_sum': 1.2,
             'day_of_year': 1,
             'day_of_month': 1,
             'month': 1,
             'year': 1991,
             'net_radiation': nan,
             'ETP': nan,
             'vpd': 0.042075189940294196})